In [1]:
# Manipulação de dados
import pandas as pd
import numpy as np

# Normalização
from sklearn.preprocessing import MinMaxScaler

# Bibliotecas de RL e ambiente
import gym
from gym import spaces

# Construção do modelo
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Flatten, Dropout
from tensorflow.keras.optimizers import Adam

# Agente de RL
from rl.agents.dqn import DQNAgent
from rl.policy import BoltzmannQPolicy
from rl.memory import SequentialMemory

# Callbacks e salvamento de modelos
from tensorflow.keras.callbacks import Callback

# Utilitários do sistema
import os
import glob


## Carregamento, preparação e Visualização dos Dados

In [3]:
# Carregar os dados
data = pd.read_csv('D:\\dados\\bar_M1_data_20-03-2023_a_31-08-2024.csv', parse_dates=['DateTime'])

# Criar uma cópia da coluna 'Close' original
data['Close_Original'] = data['Close']

# Lista de features para normalização
features = ['Open', 'High', 'Low', 'Close', 'Volume', 'PavioSuperior', 'PavioInferior', 'Corpo', 'Range',
            'D_Open', 'D_High', 'D_Low', 'D_Close', 'D_PavSup', 'D_PavInf', 'D_Corpo',
            'SMA4', 'SMA8', 'SMA12', 'SMA20', 'SMA50', 'SMA100', 'SMA200',
            'StochasticoK', 'StochasticoD', 'RSI', 'MACD', 'MACDSignal', 'MACDHistogram']

# Inicializar o scaler
scaler = MinMaxScaler()

# Aplicar a normalização
data[features] = scaler.fit_transform(data[features])

data

,DateTime,Open,High,Low,Close,Volume,PavioSuperior,PavioInferior,Corpo,Range,...,SMA100,SMA200,StochasticoK,StochasticoD,RSI,MACD,MACDSignal,MACDHistogram,Gatilho,Close_Original
0,2023-03-20 22:01:00,0.008828,0.008309,0.007667,0.007655,0.019047,0.012539,0.013453,0.581175,0.044670,...,0.001983,0.000000,0.149275,0.679894,0.322707,0.526869,0.531171,0.481194,0,14018.00
1,2023-03-20 22:02:00,0.007518,0.007240,0.007494,0.007552,0.006545,0.031348,0.015695,0.603285,0.018274,...,0.002045,0.000056,0.198658,0.635562,0.311794,0.522256,0.529060,0.474715,0,14017.25
2,2023-03-20 22:03:00,0.007449,0.006999,0.007080,0.007069,0.005564,0.018809,0.013453,0.595704,0.023350,...,0.002099,0.000109,0.115503,0.585678,0.265733,0.517060,0.526206,0.468425,0,14013.75
3,2023-03-20 22:04:00,0.007000,0.006550,0.007045,0.007207,0.003076,0.000000,0.011211,0.606443,0.011168,...,0.002153,0.000169,0.200449,0.546011,0.295303,0.513491,0.523122,0.466481,0,14014.75
4,2023-03-20 22:05:00,0.007173,0.006964,0.007391,0.007621,0.002422,0.000000,0.000000,0.610865,0.013198,...,0.002204,0.000229,0.470511,0.520501,0.376686,0.512138,0.520351,0.469130,0,14017.75
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
482873,2024-08-30 17:56:00,0.809504,0.808626,0.810326,0.809200,0.007789,0.000000,0.020179,0.597599,0.017259,...,0.814601,0.809405,0.246966,0.439103,0.359597,0.513443,0.514129,0.485388,0,19829.00
482874,2024-08-30 17:57:00,0.809228,0.808798,0.810568,0.809511,0.006480,0.012539,0.004484,0.608339,0.015228,...,0.814706,0.809487,0.467062,0.442631,0.426988,0.513505,0.513463,0.486942,0,19831.25
482875,2024-08-30 17:58:00,0.809539,0.808867,0.810637,0.809304,0.007200,0.018809,0.006726,0.598863,0.015228,...,0.814810,0.809572,0.575758,0.463263,0.395352,0.512971,0.512811,0.487054,0,19829.75
482876,2024-08-30 17:59:00,0.809366,0.808626,0.810326,0.809200,0.005367,0.012539,0.020179,0.600126,0.017259,...,0.814914,0.809656,0.564394,0.465122,0.380055,0.512314,0.512142,0.486910,0,19829.00


## 3. Criação do Ambiente de Aprendizado por Reforço

In [4]:
# 3.1 Definição da Classe do Ambiente
class TradingEnv(gym.Env):
    def __init__(self, data):
        super(TradingEnv, self).__init__()
        self.data = data.reset_index(drop=True)
        self.current_step = 0
        self.total_steps = len(self.data) - 1
        self.position = 0  # 0: sem posição, 1: comprado, -1: vendido
        self.balance = 0
        self.entry_price = 0
        self.real_rewards = []

        # Espaço de ação: 0 - manter/não fazer nada, 1 - comprar, 2 - vender, 3 - fechar posição
        self.action_space = spaces.Discrete(4)

        # Espaço de observação: vetor das features normalizadas
        self.observation_space = spaces.Box(low=0, high=1, shape=(len(features),), dtype=np.float32)

        # Variáveis para recompensas e penalidades
        self.recent_trades = []
        self.max_recent_trades = 10
        self.balance_factor = 1.0  # Fator de balanceamento ajustável

        # Variáveis para métricas
        self.total_profit = 0
        self.num_correct_trades = 0
        self.num_incorrect_trades = 0
        self.max_consecutive_wins = 0
        self.max_consecutive_losses = 0
        self.current_consecutive_wins = 0
        self.current_consecutive_losses = 0
        self.balance_over_time = []
        self.max_balance = 0
        self.drawdown = 0

#Explicação:
#self.data: DataFrame contendo os dados de mercado.
#self.current_step: Índice atual na sequência de dados.
#self.position: Posição atual (0 = sem posição, 1 = comprado, -1 = vendido).
#self.action_space: Define as ações que o agente pode tomar.
#self.observation_space: Define a forma das observações que o agente recebe.
#Variáveis para métricas: Usadas para rastrear o desempenho do agente.

### 3.2 Implementação dos Métodos Principais

In [5]:
# Reinicia o ambiente para um novo episódio.
def reset(self):
    self.current_step = 0
    self.position = 0
    self.balance = 0
    self.entry_price = 0
    self.recent_trades = []
    self.real_rewards = []

    # Resetar métricas
    self.reset_metrics()

    return self._next_observation()



### 3.2.2 Método _next_observation
Retorna a próxima observação (estado) para o agente.

In [6]:
def _next_observation(self):
    obs = self.data.loc[self.current_step, features].values
    return obs

### 3.2.3 Método step
Executa uma ação e avança o ambiente para o próximo estado.

In [7]:
def step(self, action):
    done = False
    reward = 0

    # Obter o gatilho atual
    gatilho = self.data.loc[self.current_step, 'Gatilho']

    # Preço de fechamento atual e próximo (usando 'Close_Original')
    current_price = self.data.loc[self.current_step, 'Close_Original']
    if self.current_step < self.total_steps:
        next_price = self.data.loc[self.current_step + 1, 'Close_Original']
    else:
        next_price = current_price

    # Lógica de ações
    if gatilho == 1:
        if action == 1 and self.position == 0:
            # Comprar
            self.position = 1
            self.entry_price = current_price
        elif action == 2 and self.position == 0:
            # Vender
            self.position = -1
            self.entry_price = current_price
        elif action == 3 and self.position != 0:
            # Fechar posição
            profit = (next_price - self.entry_price) * self.position
            reward = self._calculate_reward(profit)
            self.balance += profit
            self.total_profit += profit

            # Atualizar métricas de trades
            self._update_trade_metrics(profit)

            self.position = 0
            self.entry_price = 0
            self.recent_trades.append(1 if profit > 0 else -1)
            self.real_rewards.append(profit)
        else:
            # Manter posição ou não fazer nada
            pass
    else:
        # Gatilho é 0, fechar qualquer posição aberta
        if self.position != 0:
            # Fechar posição
            profit = (next_price - self.entry_price) * self.position
            reward = self._calculate_reward(profit)
            self.balance += profit
            self.total_profit += profit

            # Atualizar métricas de trades
            self._update_trade_metrics(profit)

            self.position = 0
            self.entry_price = 0
            self.recent_trades.append(1 if profit > 0 else -1)
            self.real_rewards.append(profit)
        done = True  # O episódio termina aqui

    # Limitar o tamanho da lista de trades recentes
    if len(self.recent_trades) > self.max_recent_trades:
        self.recent_trades.pop(0)

    # Atualizar saldo ao longo do tempo
    self.balance_over_time.append(self.balance)

    # Atualizar drawdown
    if self.balance > self.max_balance:
        self.max_balance = self.balance
    drawdown = self.max_balance - self.balance
    if drawdown > self.drawdown:
        self.drawdown = drawdown

    # Próxima observação
    self.current_step += 1
    if self.current_step >= self.total_steps:
        done = True

    obs = self._next_observation()

    return obs, reward, done, {}
#Explicação:
#Action Handling: Define como o ambiente reage às ações do agente.
#Recompensas e Métricas: Calcula as recompensas e atualiza as métricas após cada operação.
#Finalização do Episódio: O episódio termina quando o gatilho é 0 e não há mais dados.

## 3.3 Implementação do Sistema de Recompensas
Calcula a recompensa baseada no lucro/prejuízo e nas regras de assertividade.

In [8]:
def _calculate_reward(self, profit):
    base_reward = profit
    accuracy = self.recent_trades.count(1) / len(self.recent_trades) if self.recent_trades else 0

    # Recompensa extra por alta assertividade
    consecutive_wins = self._check_consecutive_trades(1)
    if consecutive_wins >= 3:
        base_reward += self.balance_factor * consecutive_wins

    # Penalidade por perdas consecutivas
    consecutive_losses = self._check_consecutive_trades(-1)
    if consecutive_losses >= 3:
        base_reward -= self.balance_factor * consecutive_losses

    return base_reward
#Explicação:
#base_reward: Inicia com o lucro ou prejuízo da operação.
#Recompensas Extras: Adiciona recompensas adicionais para acertos consecutivos.
#Penalidades: Subtrai penalidades para erros consecutivos.

### 3.3.2 Método _check_consecutive_trades
Verifica o número de trades consecutivos de um determinado tipo (acertos ou erros).

In [9]:
def _check_consecutive_trades(self, trade_type):
    count = 0
    for trade in reversed(self.recent_trades):
        if trade == trade_type:
            count += 1
        else:
            break
    return count


## 3.4 Implementação das Métricas de Performance
### 3.4.1 Método _update_trade_metrics
Atualiza as métricas relacionadas às operações após cada trade.

In [10]:
def _update_trade_metrics(self, profit):
    if profit > 0:
        self.num_correct_trades += 1
        self.current_consecutive_wins += 1
        self.current_consecutive_losses = 0
        if self.current_consecutive_wins > self.max_consecutive_wins:
            self.max_consecutive_wins = self.current_consecutive_wins
    elif profit < 0:
        self.num_incorrect_trades += 1
        self.current_consecutive_losses += 1
        self.current_consecutive_wins = 0
        if self.current_consecutive_losses > self.max_consecutive_losses:
            self.max_consecutive_losses = self.current_consecutive_losses
    else:
        # Trade neutro (sem lucro ou prejuízo)
        self.current_consecutive_wins = 0
        self.current_consecutive_losses = 0


### 3.4.2 Método reset_metrics

In [11]:
def reset_metrics(self):
    self.total_profit = 0
    self.num_correct_trades = 0
    self.num_incorrect_trades = 0
    self.max_consecutive_wins = 0
    self.max_consecutive_losses = 0
    self.current_consecutive_wins = 0
    self.current_consecutive_losses = 0
    self.balance_over_time = []
    self.max_balance = 0
    self.drawdown = 0


## 4. Configuração do Agente de Aprendizado por Reforço
### 4.1 Divisão dos Dados para Treinamento e Teste
Dividimos os dados em conjuntos de treinamento e teste, mantendo a ordem temporal.

In [12]:
# Dividir os dados em treinamento (80%) e teste (20%)
train_size = int(len(data) * 0.8)
train_data = data.iloc[:train_size]
test_data = data.iloc[train_size:].reset_index(drop=True)

# Criar ambientes para treinamento e teste
train_env = TradingEnv(train_data)
test_env = TradingEnv(test_data)


### 4.2 Construção do Modelo Neural com Dropout

In [13]:
# Obter o número de ações e a forma de observação do ambiente
nb_actions = train_env.action_space.n
obs_shape = train_env.observation_space.shape

# Construir o modelo
model = Sequential()
model.add(Flatten(input_shape=(1,) + obs_shape))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(nb_actions, activation='linear'))

# Resumo do modelo
print(model.summary())


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 29)                0         
                                                                 
 dense (Dense)               (None, 128)               3840      
                                                                 
 dropout (Dropout)           (None, 128)               0         
                                                                 
 dense_1 (Dense)             (None, 128)               16512     
                                                                 
 dropout_1 (Dropout)         (None, 128)               0         
                                                                 
 dense_2 (Dense)             (None, 4)                 516       
                                                                 
Total params: 20,868
Trainable params: 20,868
Non-traina

### 4.3 Configuração do Agente DQN
Configuramos o agente usando o algoritmo Deep Q-Network (DQN).

In [14]:
# Configurar a memória e a política
memory = SequentialMemory(limit=50000, window_length=1)
policy = BoltzmannQPolicy()

# Criar o agente DQN
dqn = DQNAgent(model=model, nb_actions=nb_actions, memory=memory, nb_steps_warmup=500,
               target_model_update=1e-2, policy=policy, gamma=0.99)
dqn.compile(Adam(lr=1e-3), metrics=['mae'])


TypeError: Keras symbolic inputs/outputs do not implement `__len__`. You may be trying to pass Keras symbolic inputs/outputs to a TF API that does not register dispatching, preventing Keras from automatically converting the API call to a lambda layer in the Functional Model. This error will also get raised if you try asserting a symbolic input/output directly.

## 5. Implementação dos Callbacks para Salvamento e Logging
### 5.1 Callback para Salvar os Melhores Modelos
Criamos um callback personalizado para salvar os 10 melhores modelos durante o treinamento.

In [ ]:
# Diretório para salvar os modelos
save_dir = 'model_checkpoints'
if not os.path.exists(save_dir):
    os.makedirs(save_dir)

class SaveBestModels(Callback):
    def __init__(self, save_dir, top_n=10):
        super(SaveBestModels, self).__init__()
        self.save_dir = save_dir
        self.top_n = top_n
        self.best_models = []

    def on_episode_end(self, episode, logs={}):
        total_reward = logs.get('episode_reward')
        if total_reward is not None:
            # Salvar o modelo se estiver entre os top_n
            self.best_models.append((total_reward, episode))
            self.best_models.sort(key=lambda x: x[0], reverse=True)
            self.best_models = self.best_models[:self.top_n]

            # Salvar o modelo atual se estiver entre os melhores
            if any(episode == ep for _, ep in self.best_models):
                filepath = os.path.join(self.save_dir, f'model_episode_{episode}_reward_{total_reward:.2f}.h5f')
                self.model.save_weights(filepath)
                print(f"Modelo salvo: {filepath}")


### 5.2 Callback para Logging das Métricas

In [ ]:
class MetricsLogger(Callback):
    def __init__(self, env):
        super(MetricsLogger, self).__init__()
        self.env = env  # Ambiente de treinamento
        self.episode = 0

    def on_episode_end(self, episode, logs={}):
        self.episode += 1
        total_profit = self.env.total_profit
        num_correct_trades = self.env.num_correct_trades
        num_incorrect_trades = self.env.num_incorrect_trades
        max_consecutive_wins = self.env.max_consecutive_wins
        max_consecutive_losses = self.env.max_consecutive_losses
        drawdown = self.env.drawdown

        print(f"--- Episódio {self.episode} ---")
        print(f"Ganho Real Total: {total_profit:.2f}")
        print(f"Número de Operações Acertadas: {num_correct_trades}")
        print(f"Número de Operações Erradas: {num_incorrect_trades}")
        print(f"Maior Número de Acertos Consecutivos: {max_consecutive_wins}")
        print(f"Maior Número de Erros Consecutivos: {max_consecutive_losses}")
        print(f"Drawdown Máximo: {drawdown:.2f}")
        print("----------------------------")

        # Resetar métricas no ambiente
        self.env.reset_metrics()


## 6. Treinamento do Agente

In [ ]:
# Criar instâncias dos callbacks
save_best_models = SaveBestModels(save_dir=save_dir, top_n=10)
metrics_logger = MetricsLogger(env=train_env)

# Lista de callbacks
callbacks_list = [save_best_models, metrics_logger]

# Treinar o agente
dqn.fit(train_env, nb_steps=50000, visualize=False, verbose=0, callbacks=callbacks_list)
#Explicação:
#callbacks_list: Lista de callbacks a serem usados durante o treinamento.
#nb_steps=50000: Número total de passos de treinamento.
#verbose=0: Define o nível de verbosidade (0 = silencioso).

## 7. Avaliação do Agente
### 7.1 Carregamento e Teste dos Modelos Salvos
Carregamos um dos melhores modelos salvos e avaliamos no conjunto de teste.

In [ ]:
# Listar os modelos salvos
model_files = glob.glob(os.path.join(save_dir, '*.h5f'))
model_files.sort()
print("Modelos salvos:")
for file in model_files:
    print(file)

# Selecionar o melhor modelo (por exemplo, o primeiro da lista)
best_model_path = model_files[0]

# Carregar os pesos do modelo
dqn.load_weights(best_model_path)

# Testar o agente no ambiente de teste
dqn.test(test_env, nb_episodes=1, visualize=False)


### 7.2 Análise das Métricas

In [ ]:
# Acessar as métricas do ambiente de teste
print(f"Ganho Real Total no Teste: {test_env.total_profit:.2f}")
print(f"Número de Operações Acertadas no Teste: {test_env.num_correct_trades}")
print(f"Número de Operações Erradas no Teste: {test_env.num_incorrect_trades}")
print(f"Maior Número de Acertos Consecutivos no Teste: {test_env.max_consecutive_wins}")
print(f"Maior Número de Erros Consecutivos no Teste: {test_env.max_consecutive_losses}")
print(f"Drawdown Máximo no Teste: {test_env.drawdown:.2f}")


## 8. Aplicação do Modelo em Novos Dados

In [ ]:
# Carregar novos dados
new_data = pd.read_csv('novo_arquivo.csv', parse_dates=['DateTime'])

# Manter a coluna 'Close' original
new_data['Close_Original'] = new_data['Close']

# Normalizar as features
new_data[features] = scaler.transform(new_data[features])

# Criar um novo ambiente
new_env = TradingEnv(new_data)

# Testar o agente no novo ambiente
dqn.test(new_env, nb_episodes=1, visualize=False)

# Analisar as métricas no novo ambiente
print(f"Ganho Real Total nos Novos Dados: {new_env.total_profit:.2f}")
# ... (restante das métricas) ...
